# Initialize

In [ ]:
# pip install crewai
import os
import sys
import subprocess

from crewai import Agent, Task, Crew
from crewai.tools import tool
from dotenv import load_dotenv

# The agent asks the developer a program file. 
# {Program}{Language}
# If it's a C++ code, the agent will add the FidesZKP.h library to the user's program.cpp program and will save it as program_AddedFidesZKPLib.cpp 
# If it's a Python code, the agent will add the Fides Python package to the user's program.py program and will save it as program_AddedFidesZKPLib.py 
# 
# {Processor} 
# Then, the agent asks the developer to choose a target processor including RISC-V and ARM. 
# Then, the agent compiles the code using g++ compiler for the target processor to generate an assembly file; program_AddedFidesZKP.s
# Then, the agent executes the commitmentGenerator file 
# to create program_AddedFidesZKP_ZKPGen.s, program_AddedFidesZKP_Param.json, program_AddedFidesZKP_Commitment.json
# 
# The agent returns program_AddedFidesZKP_ZKPGen.s, program_AddedFidesZKP_Param.json files to the developer.
# The agent uploads the program_AddedFidesZKP_Commitment.json on the Fides Innova blockchain using https://rpc.fidesinnova.io API and its own wallet.
# 
# The agent return a link to the explorer to show the submitted commitment file, https://explorer.fidesinnova.io/xxxxxxxxxx
# The Fides Innova explorer shows the "CommitmentManagerAIAgent" name as the transaction submitter on the blockchain. 

# --- Step 2: Load environment variables from .env file
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

# Device Config Agent

In [ ]:
# Role, Goal, Backstory, LLm instance, Tool (optional)

# deviceConfigTool = tool()
# deviceConfigAgent = Agent(role="commitmentGenerator")
# deviceConfigTask = Task()

@tool("deviceConfigTool")
def deviceConfigTool(myJson: str) -> str:
    """
    Save the input myJson (stringified JSON) into device_config.json.
    """
    import json

    try:
        parsed = json.loads(myJson)
    except json.JSONDecodeError:
        return "Error: Provided input is not valid JSON."

    with open("device_config.json", "w+") as f:
        json.dump(parsed, f, indent=2)

    return "✅ Device config file updated successfully."

deviceConfigAgent = Agent(
    role="device_config",
    goal=(
        "Create a JSON object called myJson with the following keys:\n"
        "- class: {class}\n"
        "- deviceType: {deviceType}\n"
        "- deviceIdType: {deviceIdType}\n"
        "- deviceModel: {deviceModel}\n"
        "- manufacturer: {manufacturer}\n"
        "- softwareVersion: {softwareVersion}\n"
        "- code_block: {code_block}\n"
        "Then call deviceConfigTool(myJson) to save it."
    ),
    backstory="You are responsible for creating device configs based on user inputs.",
    tools=[deviceConfigTool]
)

deviceConfigTask = Task(
    agent=deviceConfigAgent,
    description=(
        "Generate a JSON object called myJson with the specified keys and values. "
        "Then invoke deviceConfigTool(myJson) to save it to device_config.json."
    ),
    expected_output="Print a message whether the device config file is updated or not."
)

# Compiler Agent

In [ ]:
@tool("CompilerTool")
def compilerTool(program: str) -> str:
    """
    Read the user program file, recognize the program language.
    Add new lines to the program to import the FidesZKP library or package.
    Add '_AddedFidesZKP' to the program name and save it.
    Compile the program and return the assembly filename if successful.
    If compilation fails, raise an Exception to stop the crew.
    """
    filename = os.path.basename(program)
    base_name, ext = os.path.splitext(filename)
    ext = ext.lower()
    filename = base_name + ext

    if ext == ".py":
        language = "Python"
    elif ext == ".cpp":
        language = "C++"
    else:
        raise ValueError("Unsupported file extension")

    with open(program, "r") as f1:
        program_code = f1.read()

    if language == "C++":
        new_line = '#include "lib/fidesinnova.h"\n'
        if not (new_line in program_code):
            modified_code = new_line + program_code
        else:
            modified_code = program_code
        new_filename = f"{base_name}_AddedFidesZKPLib.cpp"

        with open(new_filename, "w+") as f2:
            f2.write(modified_code)

        asm_filename = f"{base_name}_AddedFidesZKPLib.s"
        command_to_execute = f"g++ -std=c++17 -S {new_filename} -o {asm_filename} -lstdc++"
        result = subprocess.run(command_to_execute.split(), capture_output=True, text=True)
        print(f"Executing command: {command_to_execute}")

        if result.returncode != 0:
            # Raise an exception to stop the crew and print error
            raise RuntimeError(
                f"Compilation failed!\nError Output:\n{result.stderr}\n"
                "Please check your code for syntax errors or missing dependencies. "
                "Fix the issues and try again."
            )

        return asm_filename

    else:
        raise NotImplementedError("Only C++ is supported for now")

CompilerAgent = Agent(
    role="Compiler",
    goal="Execute the compilerTool tool. The default {program} filename is program.cpp if it's not set by the user. If there is any error, print the error for the user and provide some suggestions to fix it. Then, do not proceed with any other agent and stop the crew.",
    backstory="",
    tools=[compilerTool],
    allow_delegation=True,
)

CompileTask = Task(
    agent=CompilerAgent,
    description="Process the user's program file {program} with compilerTool.",
    expected_output=(
        "If there is no error from the g++ compilation command, then return only the assembly output filename resulting from execution of the g++ compiler. "
        "If there is any error, print the error for the user and provide some suggestions to fix it. Then, do not proceed with any other agent and stop the crew."
    ),
)

# Commitment Generator Agent

In [ ]:
# Role, Goal, Backstory, LLm instance, Tool (optional)
# commitmentGeneratorTool = tool()
# commitmentGeneratorAgent = Agent(role="commitmentGenerator")
# commitmentGeneratorTask = Task()

# execute:
# commitmentGenerator-ubuntu program_AddedFidesZKPLib.s
# commitmentGenerator-Mac program_AddedFidesZKPLib.s

# Role, Goal, Backstory, LLm instance, Tool (optional)
@tool("operatingSystemTool")
def operatingSystemTool() -> str:
    """
    Find out what the operating system is. This must be executed to let the agent learn about the operating system name.
    """
    if sys.platform.startswith('linux'):
        # Further check for Ubuntu specifically
        if os.path.exists('/etc/lsb-release'):
            with open('/etc/lsb-release', 'r') as f:
                content = f.read()
                if 'DISTRIB_ID=Ubuntu' in content:
                    return "Ubuntu"
                else:
                    return "Linux (Other)"
        else:
             return "Linux (Likely)"
    elif sys.platform == 'darwin':
        return "macOS"
    else:
        return "Other OS"

@tool("commitmentGeneratorOnMacTool")
def commitmentGeneratorOnMacTool(program: str) -> str:
    """
    Execute ./commitmentGenerator using the output of the Compiler agent output. This execution is for macOS machines.
    """

    command_to_execute = f"./commitmentGenerator-Mac {program}"
    result = subprocess.run(command_to_execute.split(), capture_output=True, text=True)
    print(f"Executing command: {command_to_execute}")

    if result.returncode != 0:
        # Raise an exception to stop the crew and print error
        raise RuntimeError(
            f"Commitment Generation failed!\nError Output:\n{result.stderr}\n"
            "Please check your code for syntax errors or missing dependencies. "
            "Fix the issues and try again."
        )

    else:
        return result.stdout

@tool("commitmentGeneratorOnUbuntuTool")
def commitmentGeneratorOnUbuntuTool(program: str) -> str:
    """
    Execute ./commitmentGenerator using the output of the Compiler agent output. This execution is for Ubuntu machines.
    """
    command_to_execute = f"./commitmentGenerator-ubuntu {program}"
    result = subprocess.run(command_to_execute.split(), capture_output=True, text=True)
    print(f"Executing command: {command_to_execute}")

    if result.returncode != 0:
        # Raise an exception to stop the crew and print error
        raise RuntimeError(
            f"Commitment Generation failed!\nError Output:\n{result.stderr}\n"
            "Please check your code for syntax errors or missing dependencies. "
            "Fix the issues and try again."
        )

    else:
        return result.stdout

commitmentGeneratorAgent = Agent(role="commitmentGenerator", 
                                 goal = "If there is an error from the CompilerAgent output, then do nothing, do not execute any tool, and stop the crew. Otherwise, based on the return value of operatingSystemTool, find out which computer's operation system is currently running on the computer. If it's a macOS, execute commitmentGeneratorOnMacTool tool. The input for this tool is the output filename from the Compiler agent. If it's a Ubuntu, execute commitmentGeneratorOnUbuntuTool tool. The input for this tool is the output filename from the Compiler agent. Otherwise, tell the user that you support only macOS and Ubuntu platform at the moment and you do not have a tool for the current operating system.",
                                 backstory="", 
                                 tools = [operatingSystemTool, commitmentGeneratorOnMacTool, commitmentGeneratorOnUbuntuTool]
                                 )
FindingOperatingSystemTask = Task( agent = commitmentGeneratorAgent, 
                               description = "Call the operatingSystemTool with the output of the Compiler agent as the input. Then, based on the output operatingSystemTool, determine the type of operating system.", 
                               expected_output="Operating System type" 
                               )
ExecutingCommitmentGeneratorTask = Task( agent = commitmentGeneratorAgent, 
                               description = "Based on the output of the operatingSystemTool tool, choose which tool, commitmentGeneratorOnUbuntuTool or commitmentGeneratorOnLinuxTool or none, must be executed.", 
                               expected_output="If the output of the tools is successful, return in a json format with keys 'commitmentFile' and 'paramFile' and 'finalAssemblyFile'. The values are the file name from the output of the tool. If there is an error, print the error for the user and provide some suggestions to fix it. Then, do not proceed with any other agent and stop the crew." 
                               )

# Commitment Submitter Agent

In [ ]:

import json
from web3 import Web3
import datetime
import uuid
import os

# Role, Goal, Backstory, LLm instance, Tool (optional)
# The commitmentUploader agent is responsible for uploading the program_AddedFidesZKP_Commitment.json file on the Fides Innova blockchain.
# The agent will use the Fides Innova API to upload the file.
# The agent will use the wallet address and private key to sign the transaction.
# The CommitmentStorage.sol smart contract is already deployed on the Fides Innova blockchain and its source code and ABI are available in the current folder.
# The agent will use its tool function to upload the program_AddedFidesZKP_Commitment.json file on the blockchain.
# the function storeCommitment function in the smart contract will be used mainly to upload the file.
# function storeCommitment(
#     string memory commitmentId,
#     string memory nodeId,
#     string memory deviceType,
#     string memory deviceIdType,
#     string memory deviceModel,
#     string memory manufacturer,
#     string memory softwareVersion,
#     string memory commitment,
#     uint256 timestamp
# ) public returns (bool) {
#     require(!commitmentIds[commitmentId], "commitmentId already registered.");


# @tool("commitmentSubmitterTool")

def commitmentSubmitterTool(generatorAgentOutput: str):
    """Read the generatorAgentOutput from the output of the commitmentGenerator agent."""

    print("generatorAgentOutput: ", generatorAgentOutput)
    # load the generatorAgentOutput
    generatorAgentOutput = json.loads(generatorAgentOutput)
    # get the commitment file name
    commitmentFile = generatorAgentOutput["commitmentFile"]
    # get the param file name
    paramFile = generatorAgentOutput["paramFile"]
    # get the final assembly file name
    finalAssemblyFile = generatorAgentOutput["finalAssemblyFile"]
 
    # check if the param file exists
    if not os.path.exists(paramFile):
        raise FileNotFoundError(f"{paramFile} file not found. Please make sure it is in the current directory.")
    else:
        print(f"{paramFile} file found.")        
        with open(paramFile, "r") as f2:
            params = json.load(f2)
        # Check if the param data is valid
        if not isinstance(params, dict):
            raise ValueError("Invalid param file. It should be a dictionary.")
        else:  
            print(f"{paramFile} file is valid.")
    # do the same checking for the final assembly file
    if not os.path.exists(finalAssemblyFile):
        raise FileNotFoundError(f"{finalAssemblyFile} file not found. Please make sure it is in the current directory.")
    else:
        print(f"{finalAssemblyFile} file found.")
    # print to let the user know that param file and final assembly files should be used on the device
    print(f"----> Please use the {paramFile} and {finalAssemblyFile} files on the device.")


    # do the same checking for the commitment file
    # check if the commitment file exists
    if not os.path.exists(commitmentFile):
        raise FileNotFoundError(f"{commitmentFile} file not found. Please make sure it is in the current directory.")
    else:
        print(f"{commitmentFile} file found.")
    # load the commitment file
    with open(commitmentFile, "r") as f1:
        commitmentData = f1.read()
    # Check if the commitment file is a valid json file
    try:
        json.loads(commitmentData)
    except json.JSONDecodeError:
        raise ValueError("Invalid commitment file. It should be a valid json file.")
    # Check if the commitment data is valid
    if not commitmentData:
        raise ValueError("Invalid commitment data. It should not be empty.")
    # Check if the commitment data is valid
    if len(commitmentData) > 1000000:
        raise ValueError("Invalid commitment data. It should not be larger than 1MB.")
    # Check if the commitment data is valid
    if len(commitmentData) < 10:
        raise ValueError("Invalid commitment data. It should not be smaller than 10 bytes.")


    # do the same checking for the device_config.json file
    if not os.path.exists("device_config.json"):
        raise FileNotFoundError(f"device_config.json file not found. Please make sure it is in the current directory.")
    else:
        print(f"device_config.json file found.")
    # load the device_config.json file
    # Check if the device_config.json file is a valid json file
    try:
        with open("device_config.json", "r") as f2:
            device_config = json.load(f2)
    except json.JSONDecodeError:
        raise ValueError("Invalid device_config.json file. It should be a valid json file.")
    # Check if the device config data is valid
    if not isinstance(device_config, dict):
        raise ValueError("Invalid device config data. It should be a dictionary.")
    # Check if the commitment data is valid
    required_keys = [
        "deviceType",
        "deviceIdType",
        "deviceModel",
        "manufacturer",
        "softwareVersion"
    ]
    for key in required_keys:
        if key not in device_config:
            raise ValueError(f"Missing required key in device config data: {key}")
        if not isinstance(device_config[key], str):
            raise ValueError(f"Invalid value for key {key} in device config data. It should be a string.")
    # Check if the device_config are valid
    if not device_config["class"]:
        raise ValueError("Invalid class. It should not be empty.")
    if not isinstance(device_config["class"], int):
        raise ValueError("Invalid class. It should be an integer.")
    # Check if the device_config are valid
    if device_config["class"] > 30:
        raise ValueError("Invalid class. It should not be larger than 30.")
    # Check if the device_config are valid
    if device_config["class"] < 1:
        raise ValueError("Invalid class. It should not be smaller than 1.")


    # Call the polite agent to make sure all the used strings do not have an impolite or rude words and phrases. Also, the are not offensive As they are going to be registered on the blockchain forever.


    # print to let the user know that now this tool will upload the commitment file should be uploaded on the blockchain
    print(f"Now this tool will upload the {commitmentFile} file on the blockchain.")

    # load the environment variables from the .env file for the blockchain operations
    Blockchain_Name = os.getenv("Blockchain_Name")
    Blockchain_Symbol = os.getenv("Blockchain_Symbol")
    Blockchain_ID = int(os.getenv("Blockchain_ID"))
    Blockchain_API_Provider = os.getenv("Blockchain_API_Provider")
    Blockchain_Commitment_Smart_Contract_Address = os.getenv("Blockchain_Commitment_Smart_Contract_Address")

    Submitter_Wallet_Address = os.getenv("Submitter_Wallet_Address")
    print(f"Submitter_Wallet_Address: {Submitter_Wallet_Address}")
    Submitter_Wallet_Private_Key = os.getenv("Submitter_Wallet_Private_Key")


    # check if the CommitmentStorage.abi file exists
    if not os.path.exists("CommitmentStorage.abi"):
        raise FileNotFoundError("CommitmentStorage.abi file not found. Please make sure it is in the current directory.")
    else:
        print("CommitmentStorage.abi file found.")


        # upload the commitmentFile file on the Fides Innova blockchain using the Fides Innova API
        # The API endpoint is Blockchain_API_Provider
        # All the parameters are passed to the function 'storeCommitment' in the smart contract with the ABI CommitmentStorage.abi
        # The function storeCommitment in the smart contract will be used to upload the commitmentFile.

        # Connect to Fides Innova blockchain
        w3 = Web3(Web3.HTTPProvider(Blockchain_API_Provider))

        # Check if the submitter wallet address is valid
        if Submitter_Wallet_Address and not isinstance(Submitter_Wallet_Address, str):
            raise ValueError("Invalid submitter wallet address. It should be a string.")
        # Check if the wallet address is valid
        if not w3.is_address(Submitter_Wallet_Address):
            raise ValueError("Invalid wallet address.")
        # Check if the submitter wallet private key is valid
        if Submitter_Wallet_Private_Key and not isinstance(Submitter_Wallet_Private_Key, str):
            raise ValueError("Invalid submitter wallet private key. It should be a string.")
        # Check if the smart contract address is valid
        if not w3.is_address(Blockchain_Commitment_Smart_Contract_Address):
            raise ValueError("Invalid smart contract address.")
        # Check if the blockchain ID is valid
        if Blockchain_ID and not isinstance(Blockchain_ID, int):
            raise ValueError("Invalid blockchain ID. It should be an integer.")
        # Check if the blockchain name is valid
        if Blockchain_Name and not isinstance(Blockchain_Name, str):
            raise ValueError("Invalid blockchain name. It should be a string.")
        # Check if the blockchain symbol is valid
        if Blockchain_Symbol and not isinstance(Blockchain_Symbol, str):
            raise ValueError("Invalid blockchain symbol. It should be a string.")
        # Check if the blockchain API provider is valid
        if Blockchain_API_Provider and not isinstance(Blockchain_API_Provider, str):
            raise ValueError("Invalid blockchain API provider. It should be a string.")
        # Check if the blockchain commitment smart contract address is valid
        if Blockchain_Commitment_Smart_Contract_Address and not isinstance(Blockchain_Commitment_Smart_Contract_Address, str):
            raise ValueError("Invalid blockchain commitment smart contract address. It should be a string.")

        # print all the environment variables to let the user know that they are set
        print(f"Blockchain_Name: {Blockchain_Name}")
        print(f"Blockchain_Symbol: {Blockchain_Symbol}")
        print(f"Blockchain_ID: {Blockchain_ID}")
        print(f"Blockchain_API_Provider: {Blockchain_API_Provider}")
        print(f"Blockchain_Commitment_Smart_Contract_Address: {Blockchain_Commitment_Smart_Contract_Address}")

        # Load contract ABI
        with open("CommitmentStorage.abi", "r") as abi_file:
            contract_abi = json.load(abi_file)

        contract = w3.eth.contract(
            address=Web3.to_checksum_address(Blockchain_Commitment_Smart_Contract_Address),
            abi=contract_abi
        )
        
        # Check if connected to the blockchain
        if not w3.is_connected():
            raise ConnectionError("Failed to connect to the Fides Innova blockchain.")

        commitment_id = str(uuid.uuid4())
        # get the current timestamp
        now = datetime.datetime.now()

        # Prepare transaction
        # Fides Innova's native token is FDS, but gas price is still specified in gwei units for compatibility with Ethereum tooling.
        # The following line calls the storeCommitment function in the smart contract:
        tx = contract.functions.storeCommitment(
            commitment_id,
            "zkSensor",
            device_config["deviceType"],
            device_config["deviceIdType"],
            device_config["deviceModel"],
            device_config["manufacturer"],
            device_config["softwareVersion"],
            commitmentData,
            int(now.timestamp())
        ).build_transaction({
            "from": Submitter_Wallet_Address,
            "nonce": w3.eth.get_transaction_count(Submitter_Wallet_Address),
            "gas": 500000,
            "gasPrice": w3.to_wei("5", "gwei"),  # still use gwei units for gas price
            "chainId": int(Blockchain_ID) if Blockchain_ID else 1
        })

        # Sign and send transaction
        signed_tx = w3.eth.account.sign_transaction(tx, private_key=Submitter_Wallet_Private_Key)
        tx_hash = w3.eth.send_raw_transaction(signed_tx.raw_transaction)

        # Wait for transaction to be mined
        tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash, timeout=120)

        # # Check the status field: 1 = success, 0 = failure
        # if tx_receipt["status"] != 1:
        #     raise RuntimeError(f"❌ Transaction failed. View on explorer: https://explorer.fidesinnova.io/{tx_hash.hex()}")

        if tx_receipt["status"] != 1:
            # If failed, try to get the revert reason
            try:
                # Rebuild the same call as a static call to get the revert reason
                revert_reason = w3.eth.call({
                    "to": Blockchain_Commitment_Smart_Contract_Address,
                    "from": Submitter_Wallet_Address,
                    "data": tx["data"]
                }, block_identifier=tx_receipt["blockNumber"])
            except Exception as e:
                # Now decode revert reason from the error
                try:
                    # Most revert messages follow this format
                    if hasattr(e, 'args') and len(e.args) > 0 and "revert" in e.args[0]:
                        reason = e.args[0].split("revert")[-1].strip()
                        raise RuntimeError(f"❌ Transaction failed. Reason: {reason}\nExplorer: https://explorer.fidesinnova.io/{tx_hash.hex()}")
                    else:
                        raise RuntimeError(f"❌ Transaction failed without explicit revert reason.\nExplorer: https://explorer.fidesinnova.io/{tx_hash.hex()}")
                except Exception:
                    raise RuntimeError(f"❌ Transaction failed, and revert reason could not be parsed.\nExplorer: https://explorer.fidesinnova.io/{tx_hash.hex()}")

        # Transaction succeeded
        explorer_link = f"https://explorer.fidesinnova.io/{tx_hash.hex()}"
        print(f"✅ Commitment uploaded successfully! View on explorer: {explorer_link}")

    return f"✅ Commitment uploaded! View on explorer: {explorer_link}", f"✅ Transaction receipt: {tx_receipt}"



In [ ]:

commitmentSubmitterAgent = Agent(role="commitmentSubmitter", 
                                 goal="Read the commitment file from the commitmentGenerator agent, and also, read the device_config.json, then call the commitmentSubmitter tool to upload the commitment json file on the Fides Innova blockchain network.",
                                 backstory="",
                                 tools=[commitmentSubmitterTool])
commitmentSubmitterTask = Task(agent=commitmentSubmitterAgent,
                               description="Read the commitment file from the commitmentGenerator agent, and also, read the device_config.json, then call the commitmentSubmitter tool to upload the commitment json file on the Fides Innova blockchain network.",
                               expected_output="A success message indicating the commitment file has been uploaded as well as a link to the transaction on the blockchain explorer." )


In [ ]:
commitmentSubmitterTool("""{
                           "commitmentFile":"AgentAIMiniCode_AddedFidesZKPLib_commitment.json", 
                           "paramFile":"AgentAIMiniCode_AddedFidesZKPLib_param.json", 
                           "finalAssemblyFile":"AgentAIMiniCode_AddedFidesZKPLib_AddedFidesProofGen.s"
                       }""")

# Verifiable Computing Crew

In [ ]:
# Agent, Description, Expected Output   
agents = [deviceConfigAgent, CompilerAgent,  commitmentGeneratorAgent, commitmentSubmitterAgent]
tasks =  [deviceConfigTask,  CompileTask,    FindingOperatingSystemTask, ExecutingCommitmentGeneratorTask, commitmentSubmitterTask]
crew1 = Crew( agents = agents, tasks= tasks, verbose = True)

# Test

In [ ]:
def query():
    question = input("What's your program name? (default: program.cpp)")
    program_class = input("What's your program ZKP class? (default: 1)")
    deviceType = input("What's your program device type? (default: Sensor)")
    deviceIdType = input("What's your device ID type? (default: MAC)")
    deviceModel = input("What's your device model? (default: Siemens IOT2050)")
    manufacturer = input("What's your brand name? (default: Fides)")
    softwareVersion = input("What's your program/firmware version? (default: 1.0)")
    code_block = input("What's your program/firmware version? (default: [14, 15]])")

    result = crew1.kickoff({"program":question, 
                            "class": program_class or "1",
                            "deviceType": deviceType or "sensor",
                            "deviceIdType": deviceIdType or "MAC",
                            "deviceModel": deviceModel or "Siemens IOT2050",
                            "manufacturer": manufacturer or "FDS",
                            "softwareVersion": softwareVersion or "1.0",
                            "code_block": code_block or "[14, 15]"
                            })
    return result

query()